## EDL Authentication via earthaccess

In this tutorial, we demostrate how to inherit EDL authentication credentials from [earthaccess](https://earthaccess.readthedocs.io/en/stable/) and use them with PyDAP. 

There are two ways in which to use `earthaccess` to authenticate:

1. Retrieve credentials and store these locally in a `.netrc` file.
2. Inherit a request session object from `earthaccess` with EDL token header, and use it to create an optimized PyDAP session object (which may be a [CachedSession](https://requests-cache.readthedocs.io/en/stable/)).


While not strictly necessary, [earthaccess](https://earthaccess.readthedocs.io/en/stable/) hides the abstraction needed to authenticate via username/password or token-based, for Earthdata Login.



In [ ]:
import earthaccess
from pydap.client import create_session

### 1. Store EDL authentication credentials for later reuse

This is accomplished in a single line


In [ ]:
auth = earthaccess.login(strategy="interactive", persist=True) 

The credentials have been stored in a `.netrc` file. You can create a request session object and it will recover them automatically. 

In [ ]:
session = create_session()

### 2. Inherit an EDL token

This is accomplished by recovering a request.session object initialized by earthaccess. It will contain the EDL token header, and this can be used to create a `CachedSession` object that enables persistent consolidated metadata via pydap workflows.

In [ ]:
ea_session=auth.get_session()
ea_session

In [ ]:
cache_session = create_session(use_cache=True, session=ea_session)
cache_session

This `cache_session` is a [requests_cache.CachedSession](https://requests-cache.readthedocs.io/en/stable/) object that can be used with PyDAP. Internally, PyDAP will only enable caching metadata (i.e. the `.dmr` of the remote files), and the dimensions data (1D arrays). It significantly accelerates the Dataset aggregation at the collection level, when using `Xarray`, when using `pydap.client.consolidate_metadata`.

### Summary

You can use either the `session` or `cache_session` object to stream data. Each will enable you to authenticate via username/password or via token, respectively.

